In [2]:
import torch

import numpy as np
import pandas as pd

In [3]:
def accuracy(model, data_x, data_y, pct_close):
    n_items = len(data_y)
    X = torch.Tensor(data_x)  # 2-d torchensor
    Y = torch.Tensor(data_y)  # actual as 1-d torchensor
    oupt = model(X)  # all predicted as 2-d torchensor
    pred = oupt.view(n_items)  # all predicted as 1-d
    n_correct = torch.sum((torch.abs(pred - Y) < torch.abs(pct_close * Y)))
    result = (n_correct.item() * 100.0 / n_items)  # scalar
    return result

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = torch.nn.Linear(79, 61)  # 79-(61-61)-1
        self.hid2 = torch.nn.Linear(61, 61)
        self.oupt = torch.nn.Linear(61, 1)
        torch.nn.init.xavier_uniform_(self.hid1.weight)  # glorot
        torch.nn.init.zeros_(self.hid1.bias)
        torch.nn.init.xavier_uniform_(self.hid2.weight)
        torch.nn.init.zeros_(self.hid2.bias)
        torch.nn.init.xavier_uniform_(self.oupt.weight)
        torch.nn.init.zeros_(self.oupt.bias)

    def forward(self, x):
        z = torch.tanh(self.hid1(x))
        z = torch.tanh(self.hid2(z))
        z = self.oupt(z)  # no activation, aka Identity()
        return z

In [5]:
train_fp = '/home/anthony/src/school/cs570/cs570/project3/data/train_norm_num.csv'
test_fp = '/home/anthony/src/school/cs570/cs570/project3/data/test_norm_num.csv'

In [15]:
train_x = np.loadtxt(train_fp, delimiter=",", skiprows=1, usecols=range(1, 80), dtype=np.float32)
train_y = np.loadtxt(train_fp, delimiter=",", skiprows=1, usecols=[80], dtype=np.float32)
test_x_id = np.loadtxt(test_fp, delimiter=",", skiprows=1, usecols=range(0, 1), dtype=np.float32)
test_x = np.loadtxt(test_fp, delimiter=",", skiprows=1, usecols=range(1, 80), dtype=np.float32)

[0.23529412 0.         0.20766774 0.0334198  0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.6666667  0.5
 0.9492754  0.8833333  0.         0.         0.         0.
 0.         0.1225     0.         0.         0.         0.
 0.         0.         0.         0.12508859 0.         0.
 0.06421233 0.1400982  0.         0.         0.         0.
 0.11977971 0.41355932 0.         0.25923136 0.33333334 0.
 0.6666667  0.5        0.375      0.33333334 0.         0.5
 0.         0.         0.         0.         0.9965174  0.
 0.5        0.3864598  0.         0.         0.         0.
 0.11151737 0.         0.         0.         0.         0.
 0.         0.         0.         0.09090909 0.5        0.
 0.        ]


In [22]:
net = Net()

In [23]:
net = net.train()  # set training mode
bat_size = 10

loss_func = torch.nn.MSELoss()  # mean squared error
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

n_items = len(train_x)
batches_per_epoch = n_items // bat_size
max_batches = 1000 * batches_per_epoch

for b in range(max_batches):
    curr_bat = np.random.choice(n_items, bat_size, replace=False)
    X = torch.Tensor(train_x[curr_bat])
    Y = torch.Tensor(train_y[curr_bat]).view(bat_size,1)

    optimizer.zero_grad()
    oupt = net(X)
    loss_obj = loss_func(oupt, Y)
    loss_obj.backward()
    optimizer.step()

    if b % (max_batches // 10) == 0:
        print("batch = %6d" % b, end="")
        print("  batch loss = %7.4f" % loss_obj.item(), end="")
        net = net.eval()
        acc = accuracy(net, train_x, train_y, 0.15)
        net = net.train()
        print("  accuracy = %0.2f%%" % acc)


batch =      0  batch loss =  0.1742  accuracy = 7.05%
batch =  14600  batch loss =  0.0005  accuracy = 63.84%
batch =  29200  batch loss =  0.0004  accuracy = 65.89%
batch =  43800  batch loss =  0.0006  accuracy = 66.37%
batch =  58400  batch loss =  0.0012  accuracy = 65.89%
batch =  73000  batch loss =  0.0013  accuracy = 68.63%
batch =  87600  batch loss =  0.0017  accuracy = 69.45%
batch = 102200  batch loss =  0.0014  accuracy = 67.12%
batch = 116800  batch loss =  0.0003  accuracy = 68.70%
batch = 131400  batch loss =  0.0002  accuracy = 68.84%


In [24]:
torch.save(net.state_dict(), 'norm.pth')

In [25]:
net = net.eval()  # set eval mode
# acc = accuracy(net, train_x, train_y, 0.15)
# print("Accuracy on test data = %0.2f%%" % acc)

In [33]:
pred = net(torch.Tensor(test_x[0]))

In [34]:
pred

tensor([0.1139], grad_fn=<AddBackward0>)

In [10]:
loaded = Net()
loaded.load_state_dict(torch.load('norm.pth'))
loaded = loaded.eval()
loaded(torch.Tensor(train_x[0]))[0].item()

0.24004225432872772